# Postfix To NFA

In [ ]:
import shunting_yard
from regex2NFA import State,NFA
from NFA2DFA import DFA
from minDFA import MinimizedDFA
from IPython.display import Image

# Visualization Function

In [5]:
from graphviz import Digraph

def visualize_NFA(nfa):

  gra = Digraph(graph_attr={'rankdir':'LR'})


  for s in nfa.states:
      if(s.name == nfa.start.name):
        gra.node("", _attributes={'shape' : 'none'})
        gra.edge("", s.name)
      if(s.name == nfa.accepting.name):
        gra.node(s.name, _attributes={'peripheries' : '2'})
      else:
        gra.node(s.name)

  edge_set = set()
  for state in nfa.states:
    for edge in state.out_edges:

        destination = edge.dest
        edge_key = (state.name, destination.name, edge.label)
        if edge_key not in edge_set:
            edge_set.add(edge_key)
            gra.edge(state.name, destination.name, label=edge.label)

  gra.format = 'png'
  gra.render('NFA', view = True)
  return gra.source

ModuleNotFoundError: No module named 'graphviz'

# NFA To DFA

In [144]:
postfix = shunting_yard.infix2postfix('a+b')
nfa = NFA.postfix2NFA(postfix)
nfa.to_json()

dfa = DFA()
transitions,renamed_transitions=dfa.NFA2DFA(nfa)
dfa.to_json(transitions,renamed_transitions)

minimized_dfa = MinimizedDFA(dfa, renamed_transitions)
minimized_dfa.minimize()
minimized_dfa.to_json()


In [ ]:
visualize_NFA(nfa)
Image(filename='NFA.png')